#   Cours 3 Classification

##  1 Dev Neural network

In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [118]:
data = pd.read_csv("/home/kduplat/Documents/cours ML/TP_Filtrage/train.csv.zip")
np_data = np.array(data)
print(np.shape(np_data))

(42000, 785)


In [119]:
shuf_data = np_data.copy()
np.random.shuffle(shuf_data)

In [120]:
X_dev = shuf_data[:1000, 1:]
Y_dev = shuf_data[:1000, 0]

In [121]:
X_train = shuf_data[1000:, 1:]
Y_train = shuf_data[1000:, 0]

In [122]:
X_dev_norm = X_dev / 255.
X_train_norm = X_train / 255.

### Initialisation

In [123]:
def para_init(nb_neurone = 10, nb_pixel = 784):
    W0 = np.random.uniform(-0.5, 0.5, size=(nb_pixel, nb_neurone))
    W1 = np.random.uniform(-0.5, 0.5, size=(nb_neurone, nb_neurone))
    b0 = np.random.uniform(-0.5, 0.5, size=nb_neurone)
    b1 = np.random.uniform(-0.5, 0.5, size=nb_neurone)
    return W0, W1, b0, b1

def ReLU(Z):
    return np.maximum(0,Z)

def fct_softmax(Z):
    return np.exp(Z)/np.sum(np.exp(Z))

def propagation_forward(X, W0, W1, b0, b1):   
    Z0 = np.dot(W0.T, X) + b0
    A0 = ReLU(Z0)
    Z1= np.dot(W1.T,  A0) + b1
    A1 = fct_softmax(Z1)
    
    return Z0, A0, Z1, A1

def der_ReLU(Z):
    return Z > 0

def exp_result(y):
    tab = np.zeros(10)
    tab[y] = 1
    return tab

### This is applied for 1 image (number)
def propagation_backward(X, Y, A1, Z0, A0, W1):
    delta1 = A1 - exp_result(Y)
    dJdW1 = np.outer(A0, delta1.T)
    dJdb1 = delta1
    delta0 = np.dot(W1, delta1) * der_ReLU(Z0)
    dJdW0 = np.outer(X, delta0.T)
    dJdb0 = delta0
    return dJdW0, dJdW1, dJdb0, dJdb1

### dJdW0_L is  for the list of all the images (number)
def update_param(W0, W1, b0, b1, dJdW0_L, dJdW1_L, dJdb0_L, dJdb1_L, lambd):
    W0 = W0 - lambd * np.mean(dJdW0_L, axis=0)
    W1 = W1 - lambd * np.mean(dJdW1_L, axis=0)
    b0 = b0 - lambd * np.mean(dJdb0_L, axis=0)
    b1 = b1 - lambd * np.mean(dJdb1_L, axis=0)
    return W0, W1, b0, b1
    


In [124]:
test = np.array([-1, 2, 5, 0, -2])
ReLU(test)

array([0, 2, 5, 0, 0])

In [125]:

test2 = np.array([[1,2,3], 
                  [4,5,6], 
                  [7,8,9]])

minus = np.array([1, 2, 3]).reshape(-1, 1)  # Reshape pour obtenir un vecteur colonne
print(test2)
print(minus)
print(test2 - minus)

print(exp_result(5))


[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1]
 [2]
 [3]]
[[0 1 2]
 [2 3 4]
 [4 5 6]]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


###     1.3 Training the Neural Network

In [140]:
def result(A1):
    # print(A1)
    return np.argmax(A1)

def success_rate(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

def training(X_train, Y_train ,nb_epoch, W0, W1, b0, b1, lambd,nb_neurone = 10, nb_pixel = 784):
    
    dJdW0_L = np.zeros((len(X_train), nb_pixel, nb_neurone))
    dJdW1_L = np.zeros((len(X_train), nb_neurone, nb_neurone))  
    dJdb0_L = np.zeros((len(X_train), nb_neurone))
    dJdb1_L = np.zeros((len(X_train), nb_neurone))
    dpred = np.zeros(len(X_train))
    
    for i in range(nb_epoch):
        for j in range(len(X_train)):
            Z0, A0, Z1, A1 = propagation_forward(X_train[j], W0, W1, b0, b1)
            dJdW0, dJdW1, dJdb0, dJdb1 = propagation_backward(X_train[j], Y_train[j], A1, Z0, A0, W1)
            
            dJdW0_L[j] = dJdW0
            dJdW1_L[j] = dJdW1
            dJdb0_L[j] = dJdb0
            dJdb1_L[j] = dJdb1
            dpred[j] = result(A1)
            
        W0, W1, b0, b1 = update_param(W0, W1, b0, b1, dJdW0_L, dJdW1_L, dJdb0_L, dJdb1_L, lambd)
        
        if (i + 1) % 10 == 0:
            print("epoch : ", i+1)
            print("accuracy : ", success_rate(Y_train, dpred))
            print("-----------------------------------")
    
    return A1, W0, W1, b0, b1 
    


In [ ]:
epoch = 100
lambd = 1
W0, W1, b0, b1 = para_init()

A1, W0, W1, b0, b1  = training(X_train_norm, Y_train, epoch, lambd, W0, W1, b0, b1)

epoch :  10
accuracy :  0.32309756097560977
-----------------------------------
epoch :  20
accuracy :  0.5487073170731708
-----------------------------------
epoch :  30
accuracy :  0.5974634146341463
-----------------------------------
epoch :  40
accuracy :  0.5968048780487805
-----------------------------------
epoch :  50
accuracy :  0.6559756097560976
-----------------------------------
epoch :  60
accuracy :  0.7457317073170732
-----------------------------------
epoch :  70
accuracy :  0.7711707317073171
-----------------------------------
epoch :  80
accuracy :  0.7920975609756098
-----------------------------------
epoch :  90
accuracy :  0.8092439024390244
-----------------------------------
epoch :  100
accuracy :  0.8230487804878048
-----------------------------------


In [133]:
np.savez('data.npz', W0=W0, W1=W1, b0=b0, b1=b1)

In [139]:
def main(W0, W1, b0, b1):
    dpred = np.zeros(len(X_dev_norm))
    
    for j in range(len(X_dev_norm)):
        Z0, A0, Z1, A1 = propagation_forward(X_dev_norm[j], W0, W1, b0, b1)
        dpred[j] = result(A1)
        
    print("Accuracy : ", success_rate(Y_dev, dpred))

In [137]:
main()

Accuracy :  0.807


###  Bonus

In [142]:
nb_epoch = 100
lambd =  0.1
A1_2, W0_2, W1_2, b0_2, b1_2 = training(X_train_norm, Y_train , nb_epoch, W0, W1, b0, b1, lambd,nb_neurone = 10, nb_pixel = 784)

main(W0_2, W1_2, b0_2, b1_2)

epoch :  10
accuracy :  0.849609756097561
-----------------------------------
epoch :  20
accuracy :  0.8497317073170731
-----------------------------------
epoch :  30
accuracy :  0.8509512195121951
-----------------------------------
epoch :  40
accuracy :  0.8519756097560975
-----------------------------------
epoch :  50
accuracy :  0.852390243902439
-----------------------------------
epoch :  60
accuracy :  0.8530243902439024
-----------------------------------
epoch :  70
accuracy :  0.8538780487804878
-----------------------------------
epoch :  80
accuracy :  0.8546829268292683
-----------------------------------
epoch :  90
accuracy :  0.8555121951219512
-----------------------------------
epoch :  100
accuracy :  0.8564146341463414
-----------------------------------
Accuracy :  0.838
